## User Top 5 movie genre

In [14]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import IntegerType , StringType ,StructType  , StructField , FloatType , TimestampType , LongType , DateType
from pyspark.sql import functions as func
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('Friends_Analysis').getOrCreate()

In [17]:
ratings.where( func.col('user_id') == 1).show(10)

+-------+--------+
|user_id|movie_id|
+-------+--------+
|      1|      61|
|      1|     189|
|      1|      33|
|      1|     160|
|      1|      20|
|      1|     202|
|      1|     171|
|      1|     265|
|      1|     155|
|      1|     117|
+-------+--------+
only showing top 10 rows



In [3]:
ratings_schema = StructType([\
    StructField('user_id' , IntegerType() , True),
    StructField('movie_id' , IntegerType() , True)
])

ratings = spark.read.option('sep' , '\t').schema(ratings_schema).csv('ml-100k/ml-100k/u.data')

movieMetaDataSchema = StructType([\
    StructField('Id' , IntegerType() , True),\
    StructField('Name' , StringType(), True),\
    StructField('releaseDate' , DateType(), True),\
    StructField('videoReleaseDate' , DateType(), True),\
    StructField('imdb_url' , StringType(), True),\
    StructField('unknown' , StringType(), True),\
    StructField('IsAction' , IntegerType(), True),\
    StructField('IsAdventure' , IntegerType(), True),\
    StructField('IsAnimation' , IntegerType(), True),\
    StructField('IsChildren' , IntegerType(), True),\
    StructField('IsComedy' , IntegerType(), True),\
    StructField('IsCrime' , IntegerType(), True),\
    StructField('IsDocumentary' , IntegerType(), True),\
    StructField('IsDrama' , IntegerType(), True),\
    StructField('IsFantasy' , IntegerType(), True),\
    StructField('IsFilmNoir' , IntegerType(), True),\
    StructField('IsHorror' , IntegerType(), True),\
    StructField('IsMusical' , IntegerType(), True),\
    StructField('IsMystery' , IntegerType(), True),\
    StructField('IsRomance' , IntegerType(), True),\
    StructField('IsSciFi' , IntegerType(), True),\
    StructField('IsThriller' , IntegerType(), True),\
    StructField('IsWar' , IntegerType(), True),\
    StructField('IsWestern' , IntegerType(), True)
])

MovieMetaData = spark.read.option('sep' , '|').schema(movieMetaDataSchema).csv('ml-100k/ml-100k/u.item')

In [4]:
# to get the genre based on the 1 and0 value 
def getGenre( value , genre ):
    if (value ==1 ) :
        return genre

# creating genrelist as per movie id 
def createGenreList(line):
    genreList = []
    genreList.append(getGenre(line.IsAction , 'Action'))    
    genreList.append(getGenre(line.IsAdventure , 'Adventure'))
    genreList.append(getGenre(line.IsAnimation , 'Animation'))
    genreList.append(getGenre(line.IsChildren , 'Children'))
    genreList.append(getGenre(line.IsComedy , 'Comedy'))
    genreList.append(getGenre(line.IsCrime , 'Crime'))
    genreList.append(getGenre(line.IsDocumentary , 'Documentary'))
    genreList.append(getGenre(line.IsDrama , 'Drama'))
    genreList.append(getGenre(line.IsFantasy , 'Fantasy'))
    genreList.append(getGenre(line.IsFilmNoir , 'FilmNoir'))
    genreList.append(getGenre(line.IsHorror , 'Horror'))
    genreList.append(getGenre(line.IsMusical , 'Musical'))
    genreList.append(getGenre(line.IsMystery , 'Mystery'))
    genreList.append(getGenre(line.IsRomance , 'Romance'))
    genreList.append(getGenre(line.IsSciFi , 'Sci-Fi'))
    genreList.append(getGenre(line.IsThriller , 'Thriller'))
    genreList.append(getGenre(line.IsWar , 'War'))
    genreList.append(getGenre(line.IsWestern , 'Western'))

    # removing The None values 
    genreList = list (filter( lambda x : x != None  , genreList) )

    # returning movie_id and genereList
    return (line.Id  ,  genreList )

MovieMetaDataRDD = MovieMetaData.rdd

MovieMetaDataDF = MovieMetaDataRDD.map(createGenreList).toDF([ 'Id' , 'Genres'])
MovieMetaDataDF.show(10)

+---+--------------------+
| Id|              Genres|
+---+--------------------+
|  1|[Animation, Child...|
|  2|[Action, Adventur...|
|  3|          [Thriller]|
|  4|[Action, Comedy, ...|
|  5|[Crime, Drama, Th...|
|  6|             [Drama]|
|  7|     [Drama, Sci-Fi]|
|  8|[Children, Comedy...|
|  9|             [Drama]|
| 10|        [Drama, War]|
+---+--------------------+
only showing top 10 rows



In [5]:
# exploding the different genre for one movie into multiple row 
MovieWithGenreDF = MovieMetaDataDF.withColumn('genre' , func.explode(func.col('Genres'))).select( 'Id' , 'genre')

In [6]:
MovieWithGenreDF.show(10)

+---+---------+
| Id|    genre|
+---+---------+
|  1|Animation|
|  1| Children|
|  1|   Comedy|
|  2|   Action|
|  2|Adventure|
|  2| Thriller|
|  3| Thriller|
|  4|   Action|
|  4|   Comedy|
|  4|    Drama|
+---+---------+
only showing top 10 rows



In [7]:
MovieWithGenreDF.agg(func.max(func.col('Id'))).show()

+-------+
|max(Id)|
+-------+
|   1682|
+-------+



In [9]:
ratings.createOrReplaceTempView('ratings_tab')
MovieWithGenreDF.createOrReplaceTempView('MovieWithGenre')

In [12]:
# joining user data with genre details 
userGenreData = spark.sql(""" 
        select user_id , genre
        from ratings_tab join MovieWithGenre on movie_id = Id  
""") 

# sql1.show()

In [18]:
# getting cumulative data count for every user_id , genre
userGenreQumulitiveData = userGenreData.groupBy( func.col('user_id')  , func.col('genre') ).count().sort(func.col('user_id'))

# window function condition to get the first 5 best genre per user
windowFunc = Window.partitionBy(func.col('user_id')).orderBy(func.col('count').desc())
userGenreRankData = userGenreQumulitiveData.withColumn('ranks' , func.dense_rank().over(windowFunc))

## getting top 5 best genre Names per user 
userTop5GenreDetails = userGenreRankData.where( func.col('ranks') <=5).sort(func.col('user_id'))

userTop5GenreDetails.show()


+-------+--------+-----+-----+
|user_id|   genre|count|ranks|
+-------+--------+-----+-----+
|      1|   Drama|  107|    1|
|      1|  Comedy|   91|    2|
|      1|  Action|   75|    3|
|      1|Thriller|   52|    4|
|      1| Romance|   44|    5|
|      2|   Drama|   35|    1|
|      2| Romance|   16|    2|
|      2|  Comedy|   16|    2|
|      2|Thriller|   12|    3|
|      2|  Action|   10|    4|
|      2|   Crime|    9|    5|
|      3|   Drama|   22|    1|
|      3|Thriller|   21|    2|
|      3|  Action|   14|    3|
|      3|  Comedy|   12|    4|
|      3| Mystery|   11|    5|
|      4|Thriller|   11|    1|
|      4|  Action|    8|    2|
|      4|   Drama|    6|    3|
|      4|  Sci-Fi|    6|    3|
+-------+--------+-----+-----+
only showing top 20 rows

